# DeepPavlov实践

### 一　DeepPavlov数据准备

#### 下载并解压缩数据
下载命名实体识别任务（NER）的CoNLL-2003数据，将其放在文件夹data中

In [ ]:
##新建data文件夹，使用data.utils中的download_decompress
import deeppavlov
from deeppavlov.core.data.utils import download_decompress
download_decompress('http://lnsigo.mipt.ru/export/deeppavlov_data/conll2003_v2.tar.gz', 'data/')

#### 将文本数据解析为机器可读数据集
使用包含带有NE（命名实体）标签的推文的语料库.具有NER的典型文件包含有成对标记  
不同文档由-DOCSTART为开头  
不同的句子由空行分隔  
本文仅关注标记,行的第一个元素和最后一个元素,删除它们之间的POS信息  

构建一个NerDatasetReader类,提供读取数据集的功能.  
返回一个包含字段train,test,valid的字典,每个字段存储一个样本列表,每个样本由标记和标签组成,例子如下:

{'train':[...],'valid':[...],'test':[...]}

数据集中有三个独立的部分  
训练模型的训练数据  
用于评估和超参数(开始学习过程之前的参数)调整的验证数据  
用于最终评估模型的测试数据  
这三个部分分别存储在单独的txt文件里

**实现效果**  
**将'train.txt','test.txt','valid.txt'及其内容形成字典,字典由元组组成,每一个元组包含了一组关系密切的词对应的tokens和tags**

In [35]:
from pathlib import Path
class NerDatasetReader:
    def read(self,data_path,provide_pos = False):
        self.provide_pos = provide_pos
        data_parts = ['train','valid','test']
        extension = '.txt'
        #print(data_parts)
        dataset = {}
        for data_part in data_parts:#遍历每一个元素
            file_path = Path(data_path) / Path(data_part + extension)#形成文件路径
            #print(file_path)
            dataset[data_part] = self.read_file(str(file_path))#形成字典,字典的每个部分内容是xxx.txt
        return dataset
    def read_file(self,file_path):#读文件
        samples = []
        with open(file_path,'r')as rr:
            tokens = ['<DOCSTART>']#tokens标记
            pos_tags = ['0']
            tags = ['0']#tags标签
            for line in rr:
                #print(line)
                #print(self.povide)
                if 'DOCSTART' in line:#第一行
                    if len(tokens) > 1:
                        if self.provide_pos:
                            samples.append(((tokens.pos_tags),tags,))
                        else:
                            samples.append((tokens,tags))
                        tokens = []
                        pos_tags = []
                        tags = []
                elif len(line) < 2:#空行标志着一个段落的结束,一个段落为一句话,有相似成分,遇到空行将tokens和tags列表加到samples列表
                    if self.provide_pos:
                        samples.append(((tokens,tags),tags))
                    else:
                        samples.append((tokens,tags))
                        tokens = []#清空,进行下一次读取
                        pos_tags = []
                        tags = []
                else:#每一部分的读取
                    if self.provide_pos:
                        token,*_,pos,tag = line.split()
                        pos_tags.append(pos)
                    else:#只保留第一个tokens标记和最后一个tags标签
                        token,*_,tag = line.split()
                    tags.append(tag)
                    tokens.append(token)

        return samples
dataset_reader = NerDatasetReader()#实例化
dataset = dataset_reader.read('data/')#找到目录,调用方法
#print(dataset['train'])
for sample in dataset['train'][:5]:#抽取了前5个结果查看,格式化输出
    for token ,tag in zip(*sample):
        print('%s\t%s' %(token,tag))
    print()

<DOCSTART>	0

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O

Peter	B-PER
Blackburn	I-PER

BRUSSELS	B-LOC
1996-08-22	O

The	O
European	B-ORG
Commission	I-ORG
said	O
on	O
Thursday	O
it	O
disagreed	O
with	O
German	B-MISC
advice	O
to	O
consumers	O
to	O
shun	O
British	B-MISC
lamb	O
until	O
scientists	O
determine	O
whether	O
mad	O
cow	O
disease	O
can	O
be	O
transmitted	O
to	O
sheep	O
.	O



### 二　字典
训练神经网络使用两个映射  

1.{token}->{token id}：token id确定现有的标记在词向量矩阵中行的位置  
［参考资料：https://spaces.ac.cn/archives/4122 ]    　　　

2.{tag}->{tag id}:tag id用于用于产生独热向量概率分布矢量,计算神经网络输出损失度    
(稀疏向量(one-hot)，监督学习正确打标记(ground truth)) 

下面要实现的目标是实现一个类得到标记和标签与对应id的转化

**实现效果**  
**实现了标记和标号的映射关系  
在之前形成的字典的基础上,将字典里的每一个词的tokens标记和tags标签按照出现次数多少从大到小排序并从前往后标号,去除特殊词,分别存在一个  
字典和一个列表里,在之后通过判断输入的是字符串(tokens,tags标记)还是数字(标号),对应返回(标号,标记)  
根据标号大小可以判断这个词出现的频率**

In [45]:
from pathlib import Path
from collections import defaultdict,Counter
from itertools import chain
import numpy as np
class NerDatasetReader:
    def read(self,data_path,provide_pos = False):
        self.provide_pos = provide_pos
        data_parts = ['train','valid','test']
        extension = '.txt'
        dataset = {}
        for data_part in data_parts:#遍历每一个元素
            file_path = Path(data_path) / Path(data_part + extension)#形成文件路径
            dataset[data_part] = self.read_file(str(file_path))#字典
        return dataset
    def read_file(self,file_path):
        samples = []
        with open(file_path,'r')as rr:
            tokens = ['<DOCSTART>']
            pos_tags = ['0']
            tags = ['0']
            for line in rr:
                if 'DOCSTART' in line:
                    if len(tokens) > 1:
                        if self.provide_pos:
                            samples.append(((tokens.pos_tags),tags,))
                        else:
                            samples.append((tokens,tags))
                        tokens = []
                        pos_tags = []
                        tags = []
                elif len(line) < 2:
                    if self.provide_pos:
                        samples.append(((tokens,tags),tags))
                    else:
                        samples.append((tokens,tags))
                        tokens = []
                        pos_tags = []
                        tags = []
                else:
                    if self.provide_pos:
                        token,*_,pos,tag = line.split()
                        pos_tags.append(pos)
                    else:
                        token,*_,tag = line.split()
                    tags.append(tag)
                    tokens.append(token)
    
        return samples

class Vocab:
    def __init__(self,special_tokens = tuple()):
        self.special_tokens = special_tokens
        self._t2i = defaultdict(lambda:0)#字典不存在的键键值默认为１,存放标号
        self._i2t = []#存放标记/标签
    
    def fit(self,tokens):
        count = 0
        self.freqs = Counter(chain(*tokens))#形成一个字典，键是词,值是词出现的次数
        for special_token in self.special_tokens:#special_token放在最前面
            self._t2i[special_token] = count
            self._i2t.append(special_token)
            count += 1
        for token,freq in self.freqs.most_common():#去除出现在specail_tokens里面的元素，根据出现次数从多到少->从前到后标号
            if token not in self._t2i:
                self._t2i[token] = count
                self._i2t.append(token)
                count += 1
    
    def __call__(self,batch,**kwargs):#通过()方式就可以访问
        indices_batch = []
        for sample in batch:
            indices_batch.append([self[ch] for ch in sample])#self[ch]调用了__getitem__
        return indices_batch
    
    def __getitem__(self,key):#判断是字符串还是数字,返回对应的列表
        if isinstance(key,(int,np.integer)):
            return self._i2t[key]
        elif isinstance(key,str):
            return self._t2i[key]
        else:
            raise NotImplementedError("not implemented for type `{}`".format(type(key)))
     
    def __len__(self):
        return len(self._i2t)

dataset_reader = NerDatasetReader()
dataset = dataset_reader.read('data/')

special_tokens = ['<UNK>']
special_tags = ['O']

token_vocab = Vocab(special_tokens)
tag_vocab = Vocab(special_tags)

all_tokens_by_sentenses = [tokens for tokens, tags in dataset['train']]
all_tags_by_sentenses = [tags for tokens, tags in dataset['train']]

token_vocab.fit(all_tokens_by_sentenses)
tag_vocab.fit(all_tags_by_sentenses)

indices_batch = token_vocab([['Yan', 'is', 'a', 'good', 'fellow'],
 ['For', 'instance']])#生成每个单词对应的ID
print(indices_batch)

tag_indices_batch = tag_vocab([['0','0','0'],['B-ORG','I-ORG']])#生成每个标签对应的ID
print(tag_indices_batch)

token_batch = token_vocab([np.random.randint(0,512,size=10)])#10个随机的ID数生成对应ID的token
print(token_batch)


tag_batch = tag_vocab([np.random.randint(0,10,size = 5)])
print(tag_batch)#10个随机数生成对应ID的tag

10
[[0, 28, 7, 392, 2715], [1260, 0]]
[[9, 9, 9], [3, 5]]
[['1996-08-26', 'expected', 'party', 'United', 'National', 'now', 'Hong', 'before', '7', 'says']]
[['0', 'B-MISC', 'I-ORG', 'I-PER', '0']]


### 三 数据集迭代器
神经网络通常批量训练,意味着神经网络的权重更新每次基于若干个序列.需要遍历数据集然后批量的生成x和y  
这里的x类似[['Yan', 'is', 'a', 'good', 'fellow],['For','instance']](token)  
  这里的y类似[['B-PER', 'O', 'O', 'O', 'O'],['O', 'O']](tag)  
重要的概念在于批处理生成的随机化,随机化就是从数据集中随机地选取一个样本.从大量连续样本的随机数据训练是必要的,有可能得到高质量的模型   
一个批次中所有的序列需要有相同的长度 

**实现效果**  
**能够随机生成一个指定长度的向量,向量的内容为每一组相关联的词组成的列表.因为得到的向量长度可能不同,以最长的向量为准,短的向量不足的部分补0,为之后的CNN神经网络提供数据准备**

#### 弥补向量缺损长度
通过以上产生的数据发现,数据向量的长度不一致,需要生成一个二进制的01向量,1代表存在字符    的位置,0代表不存在的位置,通过填充使得所有序列的长度等于批处理序列中最长的向量长度

In [9]:
def iterator():
    data = [1,2,3]
    for d in data:
        yield d
print(iterator)

for i in iterator():
    print(i)


<function iterator at 0x7fa01f53c9d8>
1
2
3


In [48]:
from pathlib import Path
from collections import defaultdict,Counter
from itertools import chain
import numpy as np
from random import Random
class NerDatasetReader:
    def read(self,data_path,provide_pos = False):
        self.provide_pos = provide_pos
        data_parts = ['train','valid','test']
        extension = '.txt'
        dataset = {}
        for data_part in data_parts:#遍历每一个元素
            file_path = Path(data_path) / Path(data_part + extension)#形成文件路径
            dataset[data_part] = self.read_file(str(file_path))#字典
        return dataset
    def read_file(self,file_path):
        samples = []
        with open(file_path,'r')as rr:
            tokens = ['<DOCSTART>']
            pos_tags = ['0']
            tags = ['0']
            for line in rr:
                #print(line)
                #print(self.povide)
                if 'DOCSTART' in line:
                    if len(tokens) > 1:
                        if self.provide_pos:
                            samples.append(((tokens.pos_tags),tags,))
                        else:
                            samples.append((tokens,tags))
                        tokens = []
                        pos_tags = []
                        tags = []
                elif len(line) < 2:
                    if self.provide_pos:
                        samples.append(((tokens,tags),tags))
                    else:
                        samples.append((tokens,tags))
                        tokens = []
                        pos_tags = []
                        tags = []
                else:
                    if self.provide_pos:
                        token,*_,pos,tag = line.split()
                        pos_tags.append(pos)
                    else:
                        token,*_,tag = line.split()
                    tags.append(tag)
                    tokens.append(token)
        
        return samples

class Vocab:
    def __init__(self,special_tokens = tuple()):
        self.special_tokens = special_tokens
        self._t2i = defaultdict(lambda:0)#字典不存在的键键值默认为１
        self._i2t = []
    
    def fit(self,tokens):
        count = 0
        self.freqs = Counter(chain(*tokens))#形成一个字典，对应每个词出现的次数
        #print(self.freqs.items())
        for special_token in self.special_tokens:#special_tokens放在最前面
            self._t2i[special_token] = count
            self._i2t.append(special_token)
            count += 1
        for token,freq in self.freqs.most_common():#去除出现在specail_tokens里面的元素，根据出现次数从前到后标号
            if token not in self._t2i:
                self._t2i[token] = count
                self._i2t.append(token)
                count += 1
    
    def __call__(self,batch,**kwargs):
        indices_batch = []
        for sample in batch:
            indices_batch.append([self[ch] for ch in sample])
        return indices_batch
    
    def __getitem__(self,key):
        if isinstance(key,(int,np.integer)):
            return self._i2t[key]
        elif isinstance(key,str):
            return self._t2i[key]
        else:
            raise NotImplementedError("not implemented for type `{}`".format(type(key)))
     
    def __len__(self):
        return len(self._i2t)
class DatasetIterator:
    def __init__(self,data,seed:int = None):
        self.data = {
            'train':data['train'],
            'valid':data['valid'],
            'test':data['test']
        }
        self.random = Random(seed)
    def gen_batches(self,batch_size,data_type = 'train',shuffle = True):
        """
        batch_size为一次选取的批处理的样本个数,这里的一个样本是一个(tokens,tag)元组
        data_type为样本类型
        shuffle随机化
        """
        data = self.data[data_type]#这里data为数据集里的一组相关联的部分
        data_len = len(data)#列表的长度就是元组的数量
        if(data_len == 0):
            return
        
        order = list(range(data_len))#0-data_len-1
        
        if shuffle:
            self.random.shuffle(order)#将元组的下标随机化重排
        
        if batch_size < 0:
            batch_size = data_len
        
        for i in range((data_len - 1) // batch_size + 1):#按照batch_size分块选取批向量
            #迭代器将第i个且数量为batch_size个的元组里的tokens,tags分别对应打包在一起
            yield tuple(zip(*[data[o] for o in order[i * batch_size:(i + 1) * batch_size]]))
class Mask:
    def __init__(self,*args,**kwargs):
        pass
    def __call__(self,tokens_batch,**kwargs):
        """
        接收批量的tokens,返回对应长度的mask列表
        """
        batch_size = len(tokens_batch)
        max_len = max(len(utt) for utt in tokens_batch)
        mask = np.zeros([batch_size,max_len],dtype = np.float32)#返回一个用0填充的数组
        #np.zeros参考资料https://blog.csdn.net/qq_26948675/article/details/54318917
        for n,utterance in enumerate(tokens_batch):#返回一个二维数组,第一维代表是第几个列表,第二维表示列表的具体内容
            #print(n,utterance)
            mask[n,:len(utterance)] = 1
        return mask
    
dataset_reader = NerDatasetReader()
dataset = dataset_reader.read('data/')
dataset_iterator = DatasetIterator(dataset)
next(dataset_iterator.gen_batches(2, shuffle=True))
print(next(dataset_iterator.gen_batches(2, shuffle=True)))
get_mask = Mask()
get_mask([['Try', 'to', 'get', 'the', 'mask'], ['Check', 'paddings']])

((['"', 'We', 'are', 'united', 'and', 'we', 'are', 'waiting', 'for', 'the', 'government', 'to', 'decide', 'what', 'to', 'do', 'with', 'us', '.', '"'], ['Manchester', 'City', '3', '1', '0', '2', '2', '3', '3']), (['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']))


array([[1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0.]], dtype=float32)